In [10]:
import torchvision
import torchvision.transforms as T
import cv2
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [11]:
COCO_INSTANCE_CATEGORY_NAMES = ['__background__', 'person', 'bicycle', 'car',
                               'motorcycle', 'airplane',  'bus', 'train', 'truck',
                               'boat', 'traffic light', 'fire hydrant', 'N/A',
                               'stop sign', 'parking meter', 'bench', 'bird',
                               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',
                               'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
                               'umbrella', 'N/A', 'N/A', 'handbag', 'tie',
                               'suitcase', 'frisbee', 'skis', 'snowboard',
                               'sports ball', 'kite', 'baseball bat',
                               'baseball glove', 'skateboard', 'surfboard',
                               'tennis racket', 'bottle', 'N/A', 'wine glass',
                               'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
                               'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                               'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                               'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
                               'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse',
                               'remote', 'keyboard', 'cell phone', 'microwave',
                               'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
                               'book', 'clock', 'vase', 'scissors', 'teddy bear',
                               'hair drier', 'toothbrush']

In [12]:
def get_prediction(img_path, thresold):
  img = Image.open(img_path)
  transform = T.Compose([T.ToTensor()])
  img = transform(img)
  pred =model([img])

  num_predictions = len(pred)
  print("Number of prediction : ",num_predictions)
  print(pred[0]['labels'].numpy())

  pred_class = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list (pred[0]['labels'].numpy())]
  pred_boxes = [[(int(i[0]), int(i[1])), (int(i[2]), int(i[3]))] for i in list(pred[0]['boxes'])]
  pred_score = list(pred[0]['scores'].detach().numpy())

  pred_t = [pred_score.index(x) for x in pred_score if x>thresold][-1]

  pred_boxes = pred_boxes[:pred_t + 1]
  pred_class = pred_class[:pred_t + 1]

  return pred_boxes, pred_class

In [13]:
import matplotlib.pyplot as plt
from PIL import Image

In [14]:
def object_detection_api (img_path, thresold=0.5, rect_th=3, text_size=3, text_th=3):
  boxes, pred_cls = get_prediction(img_path, thresold)
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  for i in range(len(boxes)):
    cv2.rectangle(img, boxes[i][0], boxes[i][1], color=(0, 255, 0), thickness = rect_th)

    cv2.putText(img,pred_cls[i], boxes[i][0], cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th)
    # Write the prediction class
    plt.figure(figsize=(20,30))
    # display the output image
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [15]:
object_detection_api('/content/12.jpeg')

Output hidden; open in https://colab.research.google.com to view.

In [16]:
object_detection_api('/content/istockphoto-1136053333-612x612.jpg')